## JetBot Version

In [ ]:
from jupyter_clickable_image_widget import ClickableImageWidget

import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2, os, glob

In [ ]:
try:
    os.chdir('dataset_xy_5')
except:
    pass
file_list = glob.glob('*.jpg')
#global i
i = 0
#global cur_img
cur_img = file_list[i]
#print(len(file_list))
#print(file_list)

In [ ]:
#Change curImgWidget to ClickableImageWidget
#curImgWidget = widgets.Image(width=224, length=224)
curImgWidget = ClickableImageWidget(width=224, length=224)
curImgWidget.value = cv2.imencode('.jpg', cv2.imread(cur_img, cv2.IMREAD_UNCHANGED))[1].tobytes()
originalXYWidget = widgets.Image(width=224, length=224)
snapshotWidget = widgets.Image(width=224, length=224)
PreButt = widgets.Button(description='Previous Image')
NextButt = widgets.Button(description='Next Image')

count_widget = ipywidgets.IntText(description='Image Position')

In [ ]:
def annotate_img_cv2():
    global cur_img
    x = int(get_current_file_xy(cur_img)[0])
    y = int(get_current_file_xy(cur_img)[1])
    return cv2.imencode('.jpg', cv2.circle(cv2.imread(cur_img, cv2.IMREAD_UNCHANGED), (x, y), 8, (255, 0, 0), 3))[1].tobytes()

In [ ]:
def get_current_file_xy(file_name):
    xy_list = [file_name[3] + file_name[4] + file_name[5], file_name[7] + file_name[8] + file_name[9]]
    return xy_list

def edit_name(img_name, x, y, xy_len, start_x, start_y):
    #x, y = str(x), str(y)
    tem_list=[]
    tem_str=""
    for b in img_name:
        tem_list.append(b)
    for b in range(xy_len):
        tem_list[(start_x)+b]=x[b]
    for b in range(xy_len):
        tem_list[(start_y)+b]=y[b]
    tem_str="".join(tem_list)
    return tem_str

In [ ]:
def clicked(_, content, ignore):
    global cur_img, file_list
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        global i
        snapshotWidget.value = cv2.imencode('.jpg', cv2.circle(cv2.imread(file_list[i], cv2.IMREAD_UNCHANGED), (x, y), 8, (0, 255, 0), 3))[1].tobytes()
        new_name = edit_name(cur_img, ("%03d" %x), ("%03d" %y), 3, 3, 7)
        os.rename(cur_img, new_name)
        cur_img = new_name
        file_list[i] = new_name
        
curImgWidget.on_msg(clicked)

In [ ]:
#i = 0
def PreImg(change):
    global i
    global cur_img, file_list
    if (i != 0):
        i -= 1
        cur_img = file_list[i]
        curImgWidget.value = cv2.imencode('.jpg', cv2.imread(cur_img, cv2.IMREAD_UNCHANGED))[1].tobytes()
        originalXYWidget.value = annotate_img_cv2()
        count_widget.value = i

def NextImg(change):
    global i
    global cur_img, file_list
    if (i != len(file_list) - 1):
        i += 1
        cur_img = file_list[i]
        curImgWidget.value = cv2.imencode('.jpg', cv2.imread(cur_img, cv2.IMREAD_UNCHANGED))[1].tobytes()
        originalXYWidget.value = annotate_img_cv2()
        count_widget.value = i

PreButt.on_click(PreImg)
NextButt.on_click(NextImg)

In [ ]:
originalXYWidget.value = annotate_img_cv2()
display(ipywidgets.HBox([originalXYWidget, curImgWidget, snapshotWidget]), ipywidgets.HBox([PreButt, NextButt]), count_widget)